In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy import stats
import matplotlib.style as style
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew

In [2]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2
    

In [3]:
#
n='Input_A6_011'  #改路徑
#
dire= ['./BayesianRidge_Pre_0/','./BayesianRidge_Pre_1/','./BayesianRidge_Pre_2/','./BayesianRidge_Pre_3/','./BayesianRidge_Pre_4/','./BayesianRidge_Pre_5/','./BayesianRidge_Pre_6/','./BayesianRidge_Pre_7/']
# dire= ['../BayesianRidge_Pre_0/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))



Input_A6_011
./BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.0011873789203363802
test_rmse: 0.0011793076030212463
test_r2: 0.7485698201073624
feature number: 3
train_rmse: 0.001156164865557059
test_rmse: 0.001158559068342922
test_r2: 0.7570307902081052
feature number: 4
train_rmse: 0.001150165002150566
test_rmse: 0.0011536990654027642
test_r2: 0.7592450975210349
feature number: 5
train_rmse: 0.0011446746395264661
test_rmse: 0.0011517203628668956
test_r2: 0.7598274529850031
feature number: 6
train_rmse: 0.0011378939280024861
test_rmse: 0.0011484250485700167
test_r2: 0.761232098305686
feature number: 7
train_rmse: 0.0011348670130296502
test_rmse: 0.0011442370831728332
test_r2: 0.762888913734182
feature number: 8
train_rmse: 0.0011332746414664198
test_rmse: 0.0011497641706090637
test_r2: 0.7604601645431159
feature number: 9
train_rmse: 0.0011328325377942899
test_rmse: 0.0011532501709575333
test_r2: 0.7589339494532092
feature number: 10
train_rmse: 0.0011293372996746592
test_rmse: 0

train_rmse: 0.0011317790847532004
test_rmse: 0.0011768504631234735
test_r2: 0.7493809711370277
feature number: 15
train_rmse: 0.0011303855538733472
test_rmse: 0.0011858153820706089
test_r2: 0.745445108839752
feature number: 16
train_rmse: 0.0011299504266745668
test_rmse: 0.0011873436765731777
test_r2: 0.7447641846340092
feature number: 17
train_rmse: 0.0011292995847708089
test_rmse: 0.001193106016024972
test_r2: 0.7423874910017478
feature number: 18
train_rmse: 0.0011276427641675475
test_rmse: 0.0011924696204524293
test_r2: 0.7426600536741522
feature number: 19
train_rmse: 0.0011249496612165248
test_rmse: 0.0011979555502069394
test_r2: 0.7403105671536448
feature number: 20
train_rmse: 0.001121896060108658
test_rmse: 0.0012035701898832462
test_r2: 0.7379546855670547
Test_rmse_min = 1.fea_num: 9  2.rmse: 0.0011671389603255475
Test_r2_max = 1. fea_num: 9  2.r2: 0.7538257768690437
./BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.001207623107044517
test_rmse: 0.0012162499187574646
tes

train_rmse: 0.0011516376422223962
test_rmse: 0.0011705941005210625
test_r2: 0.7523255139166389
feature number: 8
train_rmse: 0.0011511895807999166
test_rmse: 0.0011718301575084143
test_r2: 0.7517609633088538
feature number: 9
train_rmse: 0.001145962117254555
test_rmse: 0.001167447420189441
test_r2: 0.7535253135110136
feature number: 10
train_rmse: 0.0011443739507484146
test_rmse: 0.0011737435418534017
test_r2: 0.7507842162413108
feature number: 11
train_rmse: 0.0011422237530820736
test_rmse: 0.0011711431957028891
test_r2: 0.7519193879332579
feature number: 12
train_rmse: 0.0011402866776380656
test_rmse: 0.0011714941478834712
test_r2: 0.751733674181059
feature number: 13
train_rmse: 0.0011398166824580663
test_rmse: 0.001175020835183379
test_r2: 0.7501282719620843
feature number: 14
train_rmse: 0.001134172483742217
test_rmse: 0.0011740120063234217
test_r2: 0.7504199764404856
feature number: 15
train_rmse: 0.0011337541930501404
test_rmse: 0.0011761173670869318
test_r2: 0.7494892839188745


In [5]:
fea_number=7
#
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A6_011.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
#
y=data.Predict
skew_data=skew_pro(data)

num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_135', 'Input_C_059', 'Input_A5_011', 'Input_C_056',
       'Input_C_053', 'Input_A3_011', 'Input_A2_011'], dtype=object)

## Linear Regression Baseline

In [6]:
def lr_rmse_ave_fea(data,fea_num):
    
    num=data.select_dtypes(exclude='object')
    numcorr=num.corr()
    cols=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_num+1).to_frame().index.to_numpy()[1:]
    y = data['Predict']
    X= data[cols]
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
    print('train_rmse:', train_rmse)
    print('test_rmse:', test_rmse)
    print('test_r2:', test_r2)
#     return train_rmse, test_rmse, test_r2

In [7]:
lr_rmse_ave_fea(skew_data,7)

train_rmse: 0.0011348670130296502
test_rmse: 0.0011442370831728332
test_r2: 0.762888913734182


## Lasso

In [8]:
def laso_alpha(x,y,parameters):
    from sklearn.linear_model import Lasso
    
    para=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        lasso=Lasso()
        lasso_reg=GridSearchCV(lasso, param_grid=parameters, scoring='neg_mean_squared_error', cv=3)
        lasso_reg.fit(X_train,y_train)
        para.append(lasso_reg.best_params_)

    print(para)
    
def laso_rmse_ave(x,y,alp):
    from sklearn.linear_model import Lasso
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        lasso_mod=Lasso(alpha=alp)
        lasso_mod.fit(X_train,y_train)
        y_lasso_train=lasso_mod.predict(X_train)
        y_lasso_test=lasso_mod.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_lasso_test)))
        r2.append(r2_score(y_test, y_lasso_test))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [9]:
parameters= {'alpha':[0.003,0.001,0.0009,0.00009,9e-06,9e-07]}
laso_alpha(skew_x,y,parameters)

[{'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}, {'alpha': 9e-05}]


In [10]:
laso_rmse_ave(skew_x,y,9e-05)

test_rmse_ave: 0.0011579877235090785
[0.0010160299642687804, 0.001104031122800272, 0.0013143893196831233, 0.0011122584767061766, 0.0011796635311421173, 0.001285406202631954, 0.0010862554547180834, 0.0013327639198425284, 0.0010804075482075287, 0.0010686716950902221]


test_r2_ave: 0.7606200169331588
[0.7983434558829974, 0.7750965835010173, 0.7395602236612229, 0.8143033113857601, 0.7801003054318818, 0.7215532185222511, 0.7112729351693647, 0.7071541212083474, 0.7761905500040447, 0.7826254645647008]


In [11]:
parameters= {'alpha':[0.003,0.001,0.0009,0.00009,9e-06,9e-07]}
laso_alpha(skew_data[title],y,parameters)

[{'alpha': 9e-07}, {'alpha': 9e-07}, {'alpha': 9e-07}, {'alpha': 9e-07}, {'alpha': 9e-06}, {'alpha': 9e-07}, {'alpha': 9e-05}, {'alpha': 9e-07}, {'alpha': 9e-06}, {'alpha': 9e-07}]


In [12]:
laso_rmse_ave(skew_data[title],y,5e-06)

test_rmse_ave: 0.0011455083316005306
[0.0009891828415996768, 0.0011267510198912306, 0.0012831399439415872, 0.0010615718659504148, 0.0011672294627778493, 0.001278031888481265, 0.0011002660021856895, 0.0013052211561260792, 0.0010858966368036843, 0.001057792498247831]


test_r2_ave: 0.7650543173640052
[0.8088596236180619, 0.7657447462702572, 0.7517968321708633, 0.8308424013530976, 0.7847115149793393, 0.7247389259500123, 0.7037768863784355, 0.7191328991165982, 0.7739106128296445, 0.7870287309737416]


## ElasticNet

In [13]:
def score(y_pred):
    return str(math.sqrt(mean_squared_error(y_test, y_pred)))

def ElasticNet_alpha(x,y,alphas,l1ratio):
    from sklearn.linear_model import ElasticNetCV
    
    alpha_=[]
    l1_ratio_=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        elastic_cv = ElasticNetCV(cv=3, max_iter=1e7, alphas=alphas,  l1_ratio=l1ratio)
        elasticmod = elastic_cv.fit(X_train, y_train.ravel())

        alpha_.append(elastic_cv.alpha_)
        l1_ratio_.append(elastic_cv.l1_ratio_)

    print('alpha')
    print(alpha_)
    print('\n')
    print('l1_ratio')
    print(l1_ratio_)
    
def ElasticNet_rmse_ave(x,y,alp,l1r):
    from sklearn.linear_model import ElasticNet
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        ElasticNet_mod=ElasticNet(alpha=alp, l1_ratio=l1r)
        ElasticNet_mod.fit(X_train,y_train)
        y_ElasticNet_train=ElasticNet_mod.predict(X_train)
        y_ElasticNet_test=ElasticNet_mod.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_ElasticNet_test)))
        r2.append(r2_score(y_test, y_ElasticNet_test))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [15]:
alphas = [0.008]

#alphas =[0.07,0.06,0.05,0.04, 0.03,0.02,0.01,0.005,0]
l1ratio = [0.02,0.01,0.03,0.04,0.06,0.07]
ElasticNet_alpha(skew_x,y,alphas,l1ratio)

alpha
[0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008]


l1_ratio
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02, 0.01]


In [16]:
ElasticNet_rmse_ave(skew_x,y,[0.008],0.012)

test_rmse_ave: 0.0011603320216119293
[0.0010193382586069367, 0.0011099963000887653, 0.0013221021884926877, 0.001115047835454972, 0.0011812196404465843, 0.0012885198499307516, 0.001083569864051246, 0.001330096372449249, 0.0010861772445131235, 0.0010672526620849774]


test_r2_ave: 0.7597295793486839
[0.7970280904659094, 0.7726596719222523, 0.7364947217448912, 0.8133707508507287, 0.7795197777746424, 0.7202026180368248, 0.7126988321412147, 0.7083252190702536, 0.7737937497323213, 0.7832023617478011]


In [17]:
alphas = [0.008]

# alphas =[0.07,0.06,0.05,0.04, 0.03,0.02,0.01,0.005,0]
l1ratio = [0.02,0.01,0.03,0.04,0.06,0.07]
ElasticNet_alpha(skew_data[title],y,alphas,l1ratio)

alpha
[0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008]


l1_ratio
[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]


In [18]:
ElasticNet_rmse_ave(skew_data[title],y,[0.008],0.01)

test_rmse_ave: 0.0011619151531082903
[0.0010200766420792506, 0.001127882071265642, 0.0013253486056828926, 0.0011113942618323368, 0.0011871222124048221, 0.0012780009522357948, 0.0010978636817376476, 0.0013228313688907894, 0.0010926846618543723, 0.001055947073099355]


test_r2_ave: 0.758928016399344
[0.7967339282807036, 0.7652742116002362, 0.7351990604150449, 0.8145917686053841, 0.7773107865069462, 0.7247522518163972, 0.7050690211223541, 0.7115027802040559, 0.7710751731791423, 0.7877711822631773]


## SVR

In [19]:
from sklearn.svm import SVR

In [21]:
# parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.2,1.3,1,1.5],'gamma': [1e-7,1e-6 1e-8,5e-7],'epsilon':[0.2,0.5,0.3,0.4]}
for i in np.arange(10):
    X_SVR_train, X_SVR_test, y_SVR_train, y_SVR_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    parameters = {'kernel': ['rbf'], 'C':[1.2,1.3,1.1,1.4,1],'gamma': [1e-7,5e-6,5e-7],'epsilon':[0.2,0.1,0.05,0]}
    clf = GridSearchCV(estimator = SVR(), param_grid = parameters , n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    clf.fit(X_SVR_train,y_SVR_train)
    print('best params')
    print (clf.best_params_)
    print('\n')

best params
{'C': 1.3, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.2, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.4, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.3, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}


best params
{'C': 1.4, 'epsilon': 0, 'gamma': 5e-06, 'kernel': 'rbf'}




In [26]:
# parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.2,1.3,1,1.5],'gamma': [1e-7,1e-6 1e-8,5e-7],'epsilon':[0.2,0.5,0.3,0.4]}
for i in np.arange(10):
    X_SVR_train, X_SVR_test, y_SVR_train, y_SVR_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    parameters = {'kernel': ['rbf'], 'C':[1.6,1.5,1.8,1.9,2],'gamma': [1e-3,1e-5,1e-4,1e-2],'epsilon':[0.1,0.05,0,0.2]}
    clf = GridSearchCV(estimator = SVR(), param_grid = parameters , n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    clf.fit(X_SVR_train,y_SVR_train)
    print('best params')
    print (clf.best_params_)
    print('\n')

best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}


best params
{'C': 1.6, 'epsilon': 0, 'gamma': 0.01, 'kernel': 'rbf'}




In [22]:
#best params {'C': 1.3, 'epsilon': 0.4, 'gamma': 1e-07, 'kernel': 'linear'}
def svr_rmse_ave(data,fea_num,model):
    y = data['Predict']
    if fea_num != None:
        num=data.select_dtypes(exclude='object')
        numcorr=num.corr()
        title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_num+1).to_frame().index.to_numpy()[1:]
        X= data[title]
    else:
        X= data.drop(['Predict'], axis=1)
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(X,y, random_state = i)
        svr = model.fit(x_train,y_train)
        y_train_pred = svr.predict(x_train)
        y_test_pred = svr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
    print('train_rmse:', train_rmse)
    print('test_rmse:', test_rmse)
    print('test_r2:', test_r2)
#     return train_rmse, test_rmse, test_r2

In [24]:
model=SVR(kernel='rbf', C=1.25, gamma= 5e-06, epsilon= 0)
svr_rmse_ave(skew_data,7,model)

train_rmse: 0.0011552584765401213
test_rmse: 0.0011443004043795043
test_r2: 0.7636970298066595


In [27]:
model=SVR(kernel='rbf', C=1.6, gamma= 0.01, epsilon= 0)
svr_rmse_ave(skew_data,None,model)

train_rmse: 0.00023250934096488344
test_rmse: 0.0012356167330097529
test_r2: 0.7242539743829243


## XGB

In [28]:
def xgb_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

### 先調 booster / n_estimators

In [46]:
#for tuning parameters
parameters_for_testing = {
#     'booster':['gbtree','gblinear','dart'],
    'booster':['dart'],
    'n_estimators':range(10,201,10),
}

other_params = {'learning_rate': 0.1, 'n_estimators': 90, 'max_depth': 5, 'min_child_weight': 1, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')


[04:02:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 70}


[04:02:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n_estimators': 60}


[04:02:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'dart', 'n

In [30]:
#for tuning parameters
parameters_for_testing = {
#     'booster':['gbtree','gblinear','dart'],
    'booster':['gbtree'],
#     'n_estimators':[50,60,70],
     'n_estimators':range(50,221,10),
}

other_params = {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')


[15:29:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 100}


[15:30:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 90}


[15:30:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 90}


[15:30:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 90}


[15:31:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster': 'gbtree', 'n_estimators': 90}


[15:31:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'booster':

### 調 max_depth / min_child_weight

In [48]:
# {'booster': 'dart', 'n_estimators': 60}

parameters_for_testing = {
#     'max_depth': [1,2,3, 4, 5, 6, 7, 8, 9, 10],
    'max_depth': [2],
    'min_child_weight': [1, 2, 3, 4, 5, 6]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:04:56] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 2}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 4}


[04:04:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_child_weight': 1}


[04:04:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 2, 'min_

In [32]:
# {'booster': 'dart', 'n_estimators': 160}

parameters_for_testing = {
#    'max_depth': [1,2,3, 4, 5, 6, 7, 8, 9, 10],
     'max_depth': [4],
    'min_child_weight': [1, 2, 3, 4, 5, 6]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 100, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[15:43:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 4, 'min_child_weight': 5}


[15:43:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 4, 'min_child_weight': 6}


[15:43:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 4, 'min_child_weight': 4}


[15:43:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 4, 'min_child_weight': 1}


[15:44:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 4, 'min_child_weight': 1}


[15:44:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'max_depth': 4, 'min_

### 調 gamma

In [49]:
# {'max_depth': 2, 'min_child_weight': 2}
parameters_for_testing = {
    'gamma': [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:05:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[04:05:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}



In [33]:
# {'max_depth': 7, 'min_child_weight': 3}
parameters_for_testing = {
    'gamma': [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 4, 'min_child_weight': 3, 'seed': 42,
'subsample': 0.8, 'colsample_bytree': 0.8, 'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[15:45:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[15:45:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[15:46:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[15:46:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[15:46:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[15:46:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}


[15:46:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'gamma': 0}



### 調 subsample / colsample_bytree

In [51]:
# {'gamma': 0}
parameters_for_testing = {
#     'colsample_bytree': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.5],
    'subsample': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] 
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'reg_alpha': 0, 'reg_lambda': 1, 'gamma':0}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:09:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.3}


[04:09:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.7}


[04:09:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.5, 'subsample': 0.9}


[04:09:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'

In [35]:
# {'gamma': 0}
parameters_for_testing = {
#     'colsample_bytree': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.4],
    'subsample': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] 
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 4, 'min_child_weight': 3, 'seed': 42,
 'reg_alpha': 0, 'reg_lambda': 1, 'gamma':0}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[15:58:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.4, 'subsample': 0.9}


[15:58:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.4, 'subsample': 0.4}


[15:58:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.4, 'subsample': 0.6}


[15:58:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.4, 'subsample': 0.9}


[15:58:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'colsample_bytree': 0.4, 'subsample': 0.6}


[15:58:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'

### reg_alpha / reg_lambda

In [52]:
# {'colsample_bytree': 0.5, 'subsample': 0.6}

parameters_for_testing = {
    'reg_alpha': [0,0.05, 0.1, 0.5 , 1, 2, 3], 
#     'reg_alpha': [0.5],     
    'reg_lambda': [0.05, 0.1, 0.5 , 1, 2, 3]
}

other_params = {'booster':'dart','learning_rate': 0.1, 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'gamma':0,'subsample':0.6,'colsample_bytree': 0.5}


for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:10:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[04:10:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[04:10:27]

In [36]:
# {'colsample_bytree': 0.9, 'subsample': 0.7}

parameters_for_testing = {
     'reg_alpha': [0,0.05, 0.1, 0.5 , 1, 2, 3], 
#    'reg_alpha': [0.05],     
    'reg_lambda': [0.05, 0.1, 0.5 , 1, 2, 3]
}

other_params = {'booster':'gbtree','learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 4, 'min_child_weight': 3, 'seed': 42,
 'gamma':0,'subsample':0.65,'colsample_bytree': 0.4}


for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[16:00:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[16:00:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[16:01:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 3}


[16:01:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.5}


[16:01:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 1}


[16:01:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'reg_alpha': 0, 'reg_lambda': 0.05}


[16:02:10

### learning rate

In [53]:
# {'reg_alpha': 0, 'reg_lambda': 0.7}

parameters_for_testing = {
   'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]
}

other_params = {'booster':'dart', 'n_estimators': 60, 'max_depth': 2, 'min_child_weight': 2, 'seed': 42,
 'gamma':0,'subsample':0.6,'colsample_bytree': 0.5,'reg_alpha': 0, 'reg_lambda': 0.7}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[04:11:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:44] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[04:11:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.2}


[04:11:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now depreca

In [37]:
# {'reg_alpha': 0.0, 'reg_lambda': 0.3}

parameters_for_testing = {
   'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2]
}

other_params = {'booster':'gbtree', 'n_estimators': 100, 'max_depth': 4, 'min_child_weight': 3, 'seed': 42,
 'gamma':0,'subsample':0.65,'colsample_bytree': 0.4,'reg_alpha': 0, 'reg_lambda': 1}

for i in np.arange(10):
    X_XGB_train, X_XGB_test, y_XGB_train, y_XGB_test = train_test_split(skew_x, y, test_size=0.2, random_state=i)
    xgb_model = XGBRegressor(**other_params)
    gsearch = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=0,scoring='neg_mean_squared_error')
    gsearch.fit(X_XGB_train,y_XGB_train)
    print('best params')
    print (gsearch.best_params_)
    print('\n')

[16:03:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[16:03:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[16:03:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[16:03:32] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.07}


[16:03:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[16:03:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best params
{'learning_rate': 0.1}


[16:03:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprec

In [131]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.1,
              booster = 'dart', 
              n_estimators = 60, 
              max_depth = 2, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.85,
              colsample_bytree = 0.4,
              reg_alpha =  0,
              reg_lambda = 0.7)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.008496031540705907
[0.007648650629856374, 0.007861428699795805, 0.008923526006532567, 0.008422016710058338, 0.007514787893473913, 0.007671313900216453, 0.009908224384169528, 0.0094945394022681, 0.008777706642056635, 0.008738121138631347]


test_r2_ave: 0.5658204873179591
[0.6994093043442213, 0.6676009790074625, 0.49424336268772506, 0.5730727307503245, 0.6729951732468282, 0.6589820348516682, 0.3470225948052349, 0.46909149956225205, 0.5156770986673376, 0.5601100952565365]


In [38]:
best_xgb_model = XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 100, 
              max_depth = 4, 
              min_child_weight = 3,
              seed = 42,
              gamma = 0,
              subsample = 0.65,
              colsample_bytree = 0.4,
              reg_alpha =  0,
              reg_lambda = 1)
xgb_ave(skew_x, y,best_xgb_model)


test_rmse_ave: 0.001183136217184553
[0.0009629032777190975, 0.001125760778901857, 0.0013516647727068972, 0.001099546434300882, 0.0012036896466071276, 0.0013048751740813984, 0.001129358453529404, 0.0014605276876066434, 0.0010821830249712575, 0.0011108529214209643]


test_r2_ave: 0.7489048265389775
[0.818880747130175, 0.7661563140745139, 0.7245788665054682, 0.818523723508132, 0.7710517285236209, 0.7130545405944282, 0.6879047425150662, 0.6483162908818181, 0.775454355621924, 0.7651269560346292]


## RandomForest

In [39]:
def rand_ave(x,y,model):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        rmse.append(math.sqrt(mean_squared_error(y_test, y_pred )))
        r2.append(r2_score(y_test, y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

In [78]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(
          criterion ='mse', 
          random_state = 42
)

In [79]:
X_RAND_train, X_RAND_test, y_RAND_train, y_RAND_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=42)

In [81]:
param_grid = {
#               'n_estimators': [50,100,200 ,500, 900],
              'n_estimators': [20,40,50,60,80],
              'max_depth': range(1,10,1),
              'min_samples_split': range(2,10,1)}

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Batch computation too fast (0.0688s.) Setting batch_size=2.
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done  14 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done  28 tasks      | elapsed:    1.1s
[Parallel(n_jobs=6)]: Done  46 tasks      | elapsed:    1.8s
[Parallel(n_jobs=6)]: Done  64 tasks      | elapsed:    2.3s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    3.1s
[Parallel(n_jobs=6)]: Done 108 tasks      | elapsed:    3.8s
[Parallel(n_jobs=6)]: Done 134 tasks      | elapsed:    4.8s
[Parallel(n_jobs=6)]: Done 160 tasks      | elapsed:    5.9s
[Parallel(n_jobs=6)]: Done 190 tasks      | elapsed:    7.3s
[Parallel(n_jobs=6)]: Done 220 tasks      | elapsed:    8.4s
[Parallel(n_jobs=6)]: Done 254 tasks      | elapsed:    9.5s
[Parallel(n_jobs=6)]: Done 288 tasks      | elapse

best params
{'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50}
best score
-6.733198261475197e-05


[Parallel(n_jobs=6)]: Done 1800 out of 1800 | elapsed:  1.4min finished


In [82]:
# {'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50} 
param_grid = {
              "min_samples_leaf": range(1,10,1),
              "max_leaf_nodes": range(2,15,1)
              }
    
clf = RandomForestRegressor(
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 4,
            min_samples_split = 4,
            random_state = 42
)

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    0.9s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    1.2s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    1.6s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    2.5s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    3.2s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    3.7s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    4.2s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:    4.8s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:    5.6s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:    6.5s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:    7.6s
[Parallel(

best params
{'max_leaf_nodes': 14, 'min_samples_leaf': 2}
best score
-6.728825706399575e-05


[Parallel(n_jobs=6)]: Done 585 out of 585 | elapsed:   24.0s finished


In [83]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 50,
            max_depth = 4,
            min_samples_split = 4,
            max_leaf_nodes = 14,
            min_samples_leaf =2,
            random_state = 42) 
rand_ave(skew_data[title], y,rand_model)

test_rmse_ave: 0.008359151931886303
[0.008046421179911343, 0.007522207654460264, 0.008970754200764595, 0.008192033878800358, 0.007261528804827015, 0.007389966886686526, 0.009434354784131698, 0.009095030814723662, 0.00923126977742949, 0.008447951337128053]


test_r2_ave: 0.5800138319443187
[0.6673317079609893, 0.6956681454215288, 0.4888757128331376, 0.5960708670050614, 0.6946648253743058, 0.6835371494626516, 0.40798747419915105, 0.5128303480153389, 0.46433192124209743, 0.5888401679289246]


In [40]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(
          criterion ='mse', 
          random_state = 42
)
X_RAND_train, X_RAND_test, y_RAND_train, y_RAND_test = train_test_split(skew_x, y, test_size=0.2, random_state=42)

In [41]:
param_grid = {
              'n_estimators': [50,30,40],
              'max_depth': range(1,10,1),
              'min_samples_split': range(2,10,1)}

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.4s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    1.0s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    1.6s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    2.8s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.6s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    4.4s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    5.2s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    6.3s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    7.1s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    8.2s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:    9.2s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:   10.4s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   11.9s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:   13.3s
[Parallel(

best params
{'max_depth': 3, 'min_samples_split': 3, 'n_estimators': 40}
best score
-1.3928260243982639e-06


In [42]:
# {'max_depth': 4, 'min_samples_split': 4, 'n_estimators': 50} 
param_grid = {
              "min_samples_leaf": range(1,10,1),
              "max_leaf_nodes": range(2,15,1)
              }
    
clf = RandomForestRegressor(
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 3,
            min_samples_split = 3,
            random_state = 42
)

gsearch = GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch.fit(X_RAND_train,y_RAND_train)

print('best params')
print (gsearch.best_params_)
print('best score')
print (gsearch.best_score_)

Fitting 5 folds for each of 117 candidates, totalling 585 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:    0.6s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    1.5s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:    2.8s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.5s
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:    4.2s
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:    5.1s
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:    6.1s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:    7.0s
[Parallel(n_jobs=6)]: Done 101 tasks      | elapsed:    8.5s
[Parallel(n_jobs=6)]: Done 116 tasks      | elapsed:   10.1s
[Parallel(n_jobs=6)]: Done 133 tasks      | elapsed:   11.7s
[Parallel(n_jobs=6)]: Done 150 tasks      | elapsed:   13.2s
[Parallel(n_jobs=6)]: Done 169 tasks      | elapsed:   15.3s
[Parallel(

best params
{'max_leaf_nodes': 3, 'min_samples_leaf': 7}
best score
-1.3714213108405075e-06


In [43]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 3,
            min_samples_split = 3,
            max_leaf_nodes = 3,
            min_samples_leaf =7,
            random_state = 42) 
rand_ave(skew_x, y,rand_model)

test_rmse_ave: 0.0011304207497722914
[0.0009631621010610734, 0.001112607593184435, 0.0013226488339561678, 0.0011080132288734412, 0.0011391472219564266, 0.0012015727501013794, 0.0010420027926743552, 0.0013154716708237892, 0.00107309187404303, 0.0010264894310488145]


test_r2_ave: 0.7721683035792953
[0.8187833662370487, 0.7715887665153962, 0.7362767752882264, 0.8157181336460945, 0.7949461018695825, 0.7566891022990665, 0.7343184959944239, 0.7147040135922187, 0.7792112132699878, 0.7994470670809081]
